In [3]:
from firedrake import *
from tools import *
from thermo_potentials import load_potential
from math import log, ceil

M_phi = 1e-3#1e-8
D = 1e-3 #m^2/s = .1 cm^2/s
interface_width = .1

x_scale = 1
c_scale = 1

Lx = 2
Ly = Lx/1
Lz = Lx/1

# Coarse mesh should have an 'appreciable' resolution. Fine mesh is scale of feature of interest
mesh_res_coarse = Lx/4
mesh_res_final = interface_width #target mesh resolution
mg_levels = ceil( log(mesh_res_coarse/mesh_res_final,2) )
print('Using {} levels of refinement'.format(mg_levels))

mesh = BoxMesh(round(Lx/mesh_res_coarse), round(Ly/mesh_res_coarse), round(Lz/mesh_res_coarse), Lx/x_scale, Ly/x_scale, Lz/x_scale, reorder=True)
#mesh = BoxMesh(round(Lx/mesh_res_final), round(Ly/mesh_res_final), round(Lz/mesh_res_final), Lx/x_scale, Ly/x_scale, Lz/x_scale, reorder=True)

#mesh = RectangleMesh(round(Lx/mesh_res_coarse), round(Ly/mesh_res_coarse), Lx/x_scale, Ly/x_scale)

hierarchy = MeshHierarchy(mesh, mg_levels)
mesh = hierarchy[-1]

# utility function to help with non-dimensionalization
def gr(x):
    return grad(x)/x_scale

#n - number of species, m = number of phases
n = 2
m = 3

xmesh = SpatialCoordinate(mesh)
x = xmesh*x_scale

V_phase = VectorFunctionSpace(mesh, "CG", 1, dim = m-1, name="phases")
V_species = VectorFunctionSpace(mesh, "CG", 1, dim=n, name ="species")
V = MixedFunctionSpace([V_species, V_phase])

U = Function(V)
dU = TrialFunction(V)
test_U = TestFunction(V)
test_c, test_phase = split(test_U)

cmesh, phase = split(U)

c = c_scale*cmesh

#Assemble full vector of phi and p_phase
phi = [p for p in phase]+[1-sum(phase)]
p_phase = [p**3*(6*p**2-15*p+10) for p in phi]
ps = as_vector(p_phase)

# Build multiphase energy -> to be moved to thermo potential.
def multiphase(p, interface_width):
    def antisymmetric_gradient(pa, pb):
        return 3*(interface_width**2*( pa*gr(pb) - pb*gr(pa) )**2 + pa**2*pb**2)
    return [antisymmetric_gradient(p[i], p[j]) for i in range(len(p)) for j in range(i)]
interface_area =  multiphase(phi, interface_width)

# pa = phi[0]
# pb = phi[1]
# pc = phi[2]
# a = 50
# interface_energy = 5000*3*(interface_width**2*( pa*gr(pb) - pb*gr(pa) )**2 + pa**2*pb**2*(1+a*pc**2)
#                     + interface_width**2*( pc*gr(pb) - pb*gr(pc) )**2 + pc**2*pb**2*(1+a*pa**2)
#                     + interface_width**2*( pc*gr(pa) - pa*gr(pc) )**2 + pc**2*pa**2*(1+a*pb**2))
interface_energy = inner(as_vector([5000,5000,5000]), as_vector(interface_area))

#Load potential
pot = load_potential('binary_3phase_elastic')
response = pot.grad([ci for ci in c]+p_phase)   #Fixme - shouldn't be negative
mu = as_vector(response[:n])
P = as_vector(response[n:])
print('Thermodynamic driver forces loaded')

# build diffusion equation
J =  -D*gr(mu)
F_diffusion = inner(J, gr(test_c))*dx
F_diffusion = 1/c_scale*F_diffusion

# build phase field equaiton
F_phase_bulk = -M_phi*inner(P, derivative(ps, phase, test_phase))*dx
F_phase_interface = -M_phi*derivative( interface_energy, phase, test_phase)*dx
F_phase = F_phase_bulk + F_phase_interface

# ~~~ Initial conditions ~~~ #
# phase initial conditions
def create_bubble(centre, radius):
    centre = as_vector(centre)
    r = sqrt(inner(x-centre, x-centre))
    return .5*(1.-tanh((r-radius)/(2.*interface_width)))

p0 = create_bubble( [.2*Lx, .2*Lx, 0], .09*Lx)
p1 = create_bubble( [.8*Lx, .8*Lx, 0], .09*Lx)
p2 = create_bubble( [.2*Lx, .8*Lx, 0], .09*Lx)
p3 = create_bubble( [.8*Lx, .2*Lx, 0], .09*Lx)
#p_total =  p0+p1 etc
#p_total = min/max()

#U.sub(1).interpolate(as_vector([p0,p1]))


Using 3 levels of refinement
Thermodynamic driver forces loaded


In [29]:
print(phi[0])

w⃗₁₆[2]


In [12]:
ic = (1/(1+2.71**(-2.0*50.0*(x[2]-0.1)))*(x[2]**(0.1)))/Lx *mesh_res_coarse
U.sub(1).interpolate(as_vector([p0,p1]))


Coefficient(WithGeometry(IndexedProxyFunctionSpace(<firedrake.mesh.MeshTopology object at 0x7f68289071c0>, VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=2), name='phases', index=1, component=None), Mesh(VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=3), 15)), 20)

In [11]:
u = as_vector([p0,p1])

#make 2 big bubbles of 2 phases
#2 problems: 1)multiple buibbles of same phase
#            2) intersection of bubbles of different phases

print(u)

[0.5 * (1.0 + -1 * tanh((-0.18 + sqrt((x + ({ A | A_{i_{62}} = -1 * ([0.4, 0.4, 0])[i_{62}] })) : (x + ({ A | A_{i_{63}} = -1 * ([0.4, 0.4, 0])[i_{63}] })))) / 0.2)), 0.5 * (1.0 + -1 * tanh((-0.18 + sqrt((x + ({ A | A_{i_{64}} = -1 * ([1.6, 1.6, 0])[i_{64}] })) : (x + ({ A | A_{i_{65}} = -1 * ([1.6, 1.6, 0])[i_{65}] })))) / 0.2))]


In [27]:
print(min(U.sub(1),1))

ValueError: Expecting scalar arguments.

In [8]:
min(1,2)

1

In [1]:
p0

NameError: name 'p0' is not defined

In [8]:
pot = load_potential('binary_and_stoichiometric_2phase_elastic')

Building potential
y0 =  [    0.79     0.21     0.29     0.21      0.5]


In [6]:
from tools import *
from thermo_potentials import load_potential


firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


In [5]:
ot = load_potential('quad_potential')

Building potential
0
1
hello
hello
f =  -137600*c0 - 5972200*c1**2
2
3


<lambdifygenerated-25>:3: RuntimeWarning: invalid value encountered in log
  return -137600*c0_a_a + c0_b_b*(11639.6*log(c0_b_b/(c0_b_b + c1_b_b)) - 3594.37) - 5972200*c1_a_a**2 + c1_b_b*(11639.6*log(c1_b_b/(c0_b_b + c1_b_b)) - 238888) + (500*c0_a_a + 500*c1_a_a)*log(V_a_a/(c0_a_a + c1_a_a))**2 + (500*c0_b_b + 500*c1_b_b)*log(V_b_b/(c0_b_b + c1_b_b))**2


y0 =  [     0.5      0.5     0.25     0.25     0.25     0.25]


/home/george/phase_field/thermo_potentials/systems/matrix_methods.py:70: RuntimeWarning: invalid value encountered in sqrt
  svi = np.diag(M)**.5


ValueError: array must not contain infs or NaNs

In [6]:
from thermo_potentials.phases.sympy_components import  build_deposit_quad
build_deposit_quad([-137600,-238888], T = 1400 , rho = 1, kappa = 1000, phase_id = "a")

hello
hello
f =  -137600*c0 - 5972200*c1**2


In [15]:
from sympy import symbols, ln, Matrix, zeros, ones
from thermo_potentials import sympy_potential

mu0 = [-137600]
T = 1400
rho = 1
kappa = 1000
phase_id = "a"
vi = None


In [16]:
n = len(mu0)

cs = Matrix(symbols('c:{}'.format(n)))
print("cs0 = ", cs)
V = symbols('V')
print("hello")
f = mu0[0]*cs[0] + mu0[1]*(5*cs[1])**2
print("hello")
print("f = ", f)
if kappa is not None:
    # kappa is provided, apply a neo-Hookean elasticity model
    if vi is None:
        #If no vi is entered, take 1/rho
        vi = 1/rho
    f += add_hyperelastic(kappa, vi, cs, V)
#if kappa is None by vi is not None
# Implement lattice constraint

cs0 =  Matrix([[c0]])
hello


IndexError: list index out of range

In [4]:
from thermo_potentials.systems import equil_partition, quadratic
from thermo_potentials.systems.collections import collect_sympy_phases
from thermo_potentials.phases.sympy_components import build_ideal_solution_elastic, build_deposit_quad


    
# a_b = build_ideal_solution_elastic([-3594.37,-238888], rho = 1, kappa = 1000, phase_id = "b")
#a_b = build_ideal_solution_elastic([184014,-238888], rho = 1, kappa = 1000, phase_id = "b")
print("0")
a_b = build_ideal_solution_elastic([-10479,-22026], T = 1300 ,rho = 1, kappa = 1000, phase_id = "b")
print("1")
#a_a = build_deposit_quad([-12968,-22026], T = 1300 , rho = 1, kappa = 1000, phase_id = "a")

a_a = build_ideal_solution_elastic([-12968], T =1300, rho = 1, kappa = 1000, phase_id = "b")

# a_b = build_ideal_solution_elastic([-6097,-4645] ,rho = 1, kappa = 1000, phase_id = "b")

# a_a = build_deposit_quad([-7820,-4645], rho = 1, kappa = 1000, phase_id = "a")
    
print("2")

a_full = collect_sympy_phases([a_a, a_b], rename=True)
print("3")
x = .5

0
ideal f =  c0*(10808.2*log(c0/(c0 + c1)) - 10479) + c1*(10808.2*log(c1/(c0 + c1)) - 22026) + 25000
1
ideal f =  25000 - 12968*c0
2
3


In [5]:
_, y0, _, _ = equil_partition.equil_partition(a_full,['c0', 'c1', 'V'],[1-x, x, 1])
    



<lambdifygenerated-16>:3: RuntimeWarning: invalid value encountered in log
  return 500*c0_b_a*log(V_b_a/c0_b_a)**2 - 12968*c0_b_a + c0_b_b*(10808.2*log(c0_b_b/(c0_b_b + c1_b_b)) - 10479) + c1_b_b*(10808.2*log(c1_b_b/(c0_b_b + c1_b_b)) - 22026) + (500*c0_b_b + 500*c1_b_b)*log(V_b_b/(c0_b_b + c1_b_b))**2 + 50000


In [6]:
y0

array([0.36035395, 0.88189943, 0.28337411, 0.58000572, 0.62112669])

In [4]:
a_quad = quadratic.quadratic_collection(a_full, ['c0', 'c1', 'V_a', 'V_b'], y0 = y0)
    

/home/george/phase_field/thermo_potentials/systems/matrix_methods.py:70: RuntimeWarning: invalid value encountered in sqrt
  svi = np.diag(M)**.5


ValueError: array must not contain infs or NaNs